# WRMSSE calculations without overhead

This notebook is based on amazing [for_Japanese_beginner(with WRMSSE in LGBM))](https://www.kaggle.com/girmdshinsei/for-japanese-beginner-with-wrmsse-in-lgbm) and [RMSE and WRMSSE of a submission](https://www.kaggle.com/chameleontk/rmse-and-wrmsse-of-a-submission)

Custom loss function requires quick calculations of WRMSSE. This notebook attempts to make a quick and clear WRMSEE calculation function with pickled S,W weights and pickled csr_matrix for swift rollups.

Note: Difference in rolled up vectors is equal to their rolled up difference:

\begin{equation}
 Y\times M - \hat{Y}\times M= (Y-\hat{Y}) \times M = D
\end{equation}

The rest of the calculations are the same:

\begin{equation}
WRMSSE = \sum_{i=1}^{42840} \left(\frac{W_i}{\sqrt{S_i}} \times \sqrt{\sum{(D)^2}}\right)
\end{equation}

Note that the real weights are W/sqrt(S) this is important for weights evaluations. Besides a single precalulated weight can be used for faster calculations.
Similar stuff in code:

```
roll_diff = rollup(preds.values-y_true.values)

SW = W/np.sqrt(S)

score = np.sum(
                np.sqrt(
                    np.mean(
                        np.square(roll_diff)
                            ,axis=1)) * SW)
```

Where S are weights based on sequence length, W are weights based on sales in USD for the 28 days.


PS: The S and W weights has been compared with well tested [wrmsse-evaluator](https://www.kaggle.com/dhananjay3/wrmsse-evaluator-with-extra-features) and the original weights. Please let me know in the comments if you spot any mistakes.

PPS: Please note: I have made a tiny mistake in WRMSSE function: should be /12 not x12 at the end. Updated.

In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error
from scipy.sparse import csr_matrix
import gc

import os
path='/Users/x644435/Documents/Private/Kaggle/M5'
os.chdir(path)
!ls

M5-Competitors-Guide-Final-10-March-2020.docx
M5-Competitors-Guide-Final-10-March-2020.pdf
PyProg
README.md
RProg
adata
catboost_info
rawdata
results
sw_df.pkl
us-natural-disaster-declarations
~$-Competitors-Guide-Final-10-March-2020.docx


In [2]:
# Memory reduction helper function:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns: #columns
        col_type = df[col].dtypes
        if col_type in numerics: #numerics
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

# WRMSSE

If you just need to calculate WRMSEE with default weights S, W, simply load them and use the function below.

### Load wieghts for WRMSSE calculations:

In [4]:
# Define fold pass here:
file_pass = 'adata/'# '/kaggle/input/fast-wrmsse-and-sw-frame/'

# Load S and W weights for WRMSSE calcualtions:
sw_df = pd.read_pickle(file_pass+'sw_df.pkl')
S = sw_df.s.values
W = sw_df.w.values
SW = sw_df.sw.values

# Load roll up matrix to calcualte aggreagates:
roll_mat_df = pd.read_pickle(file_pass+'roll_mat_df.pkl')
roll_index = roll_mat_df.index
roll_mat_csr = csr_matrix(roll_mat_df.values)
del roll_mat_df

### Functions for WRMSSE calculations:

In [5]:
# Function to do quick rollups:
def rollup(v):
    '''
    v - np.array of size (30490 rows, n day columns)
    v_rolledup - array of size (n, 42840)
    '''
    return roll_mat_csr*v #(v.T*roll_mat_csr.T).T


# Function to calculate WRMSSE:
def wrmsse(preds, y_true, score_only=False, s = S, w = W, sw=SW):
    '''
    preds - Predictions: pd.DataFrame of size (30490 rows, N day columns)
    y_true - True values: pd.DataFrame of size (30490 rows, N day columns)
    sequence_length - np.array of size (42840,)
    sales_weight - sales weights based on last 28 days: np.array (42840,)
    '''
    
    if score_only:
        return np.sum(
                np.sqrt(
                    np.mean(
                        np.square(rollup(preds.values-y_true.values))
                            ,axis=1)) * sw)/12 #<-used to be mistake here
    else: 
        score_matrix = (np.square(rollup(preds.values-y_true.values)) * np.square(w)[:, None])/ s[:, None]
        score = np.sum(np.sqrt(np.mean(score_matrix,axis=1)))/12 #<-used to be mistake here
        return score, score_matrix

### Load fake predictions:

In [16]:
# Predictions:
sub = pd.read_csv('results/2020-05-13_submission.csv')
sub = sub[sub.id.str.endswith('validation')]
sub.drop(['id'], axis=1, inplace=True)

DAYS_PRED = sub.shape[1]    # 28
print(sub.shape)
sub.head()

(30490, 28)


,F1,F2,F3,F4,F5,F6,F7,F8,F9,F10,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e-24,1.000000e-24,1.000000e-24,...,1.000000e-24,1.000000e-24,1.000000e-24,1.000000e-24,1.000000e-24,1.000000e-24,1.000000e-24,1.000000e-24,1.000000e-24,1.000000e-24
1,1.000000e-24,1.000000e-24,1.000000e-24,1.000000e-24,1.000000e-24,1.000000e-24,1.000000e-24,1.000000e-24,1.000000e-24,1.000000e-24,...,1.000000e-24,1.000000e-24,1.000000e-24,1.000000e-24,1.000000e-24,1.000000e-24,1.000000e-24,1.000000e-24,1.000000e-24,1.000000e-24
2,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e-24,1.000000e-24,1.000000e-24,...,1.000000e-24,1.000000e-24,1.000000e-24,1.000000e-24,1.000000e-24,1.000000e-24,1.000000e-24,1.000000e-24,1.000000e-24,1.000000e-24
3,2.000000e+00,2.000000e+00,1.000000e+00,1.000000e+00,2.000000e+00,3.000000e+00,2.000000e+00,1.000000e-24,1.000000e-24,1.000000e-24,...,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e-24,1.000000e-24,1.000000e-24,1.000000e+00,1.000000e+00,1.000000e-24
4,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,2.000000e+00,1.000000e-24,1.000000e-24,1.000000e-24,...,1.000000e-24,1.000000e-24,1.000000e-24,1.000000e-24,1.000000e-24,1.000000e-24,1.000000e-24,1.000000e-24,1.000000e-24,1.000000e-24


### Load ground truth:

In [13]:
# Ground truth:
sales = pd.read_csv('rawdata/sales_train_validation.csv')

dayCols = ["d_{}".format(i) for i in range(1900-DAYS_PRED, 1900)]
y_true = sales[dayCols]

### Calculate score:
If you just need the score, set Score_only = True for slightly faster calculations.

In [15]:
score = wrmsse(sub, y_true, score_only=True)
score

5.253012490100613

In [61]:
%%timeit -n 100 -r 5
# n - execute the statement n times 
# r - repeat each loop r times and return the best

score1, score_matrix = wrmsse(sub, y_true)

20 ms ± 103 µs per loop (mean ± std. dev. of 5 runs, 100 loops each)


### Score df for visualizations:
score_matrix is only needed for EDA and visualizations.

In [62]:
score = wrmsse(sub, y_true, score_only=True)
score

5.288906059212606

In [63]:
score1, score_matrix = wrmsse(sub, y_true)
score_df = pd.DataFrame(score_matrix, index = roll_index)
score_df.reset_index(inplace=True)
score_df.head()
score1

5.288906059212606